Packages

In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
import joblib
import matplotlib.pylab as pl
import seaborn as sns
import shap
from scipy.stats import pearsonr

In [11]:
rand_seed = 0

Reading the data

In [12]:
train_data = pd.read_csv("./Data/cleaned_train.csv")
test_data = pd.read_csv("./Data/cleaned_test.csv")
target = pd.read_csv("./Data/cleaned_target.csv")

target.set_index("Id", inplace = True)

Label encoding the features

In [13]:
enc = LabelEncoder()
train_result = {}
test_result = {}
for col in train_data.columns:
    if (train_data[col].dtype == "int" or train_data[col].dtype == "float"):
        train_result[col] = train_data[col]
        test_result[col] = test_data[col]
    else:
        train_result[col] = pd.Series(enc.fit_transform(train_data[col]))
        test_result[col] = pd.Series(enc.transform(test_data[col]))
train_features = pd.DataFrame(train_result)
test_features = pd.DataFrame(test_result)

train_features.set_index("Id", inplace = True)
test_features.set_index("Id", inplace = True)

XGBoost with early stopping

In [14]:
np.random.seed(rand_seed)
train_features_grid, train_features_validation, target_grid, target_validation = train_test_split(train_features, target, test_size = 0.1)

In [15]:
xgb = XGBRegressor(n_jobs = -1, booster = "gbtree", n_estimators = 1000, colsample_bytree = 0.5)
eta_range = np.linspace(0.01, 0.5, 50, endpoint = True)
max_depth_range = range(1, 7)
xgb_params = {'max_depth': max_depth_range, 'eta': eta_range}
gs_xgb = GridSearchCV(xgb, xgb_params, cv = 10, return_train_score = True)

In [16]:
np.random.seed(rand_seed)
fit_params={"early_stopping_rounds" : 20, 
            "eval_metric" : "mae", 
            "eval_set" : [[train_features_validation, target_validation]]}
%time gs_xgb.fit(train_features_grid, target_grid,verbose = 0, **fit_params)

CPU times: user 46min 55s, sys: 4min 13s, total: 51min 8s
Wall time: 6min 32s


GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster='gbtree',
                                    colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=0.5,
                                    gamma=None, gpu_id=None,
                                    importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=1000, n_...
                                    tree_method=None, validate_parameters=None,
                                    verbosity=None),
             param_grid={'eta': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19,

In [17]:
gs_xgb.best_score_

0.9089025196588716

Second round, narrowing in on eta

In [18]:
xgb = XGBRegressor(n_jobs = -1, booster = "gbtree", n_estimators = 1000, colsample_bytree = 0.5, max_depth = gs_xgb.best_params_["max_depth"])
eta_mid = gs_xgb.best_params_["eta"]
eta_range = np.linspace(eta_mid - 0.01, eta_mid + 0.01, 200, endpoint = False)
xgb_params = {'eta': eta_range}
gs_xgb = GridSearchCV(xgb, xgb_params, cv = 10, return_train_score = True)

In [19]:
np.random.seed(rand_seed)
fit_params={"early_stopping_rounds" : 20, 
            "eval_metric" : "mae", 
            "eval_set" : [[train_features_validation, target_validation]]}
%time gs_xgb.fit(train_features_grid, target_grid,verbose = 0, **fit_params)

CPU times: user 25min 40s, sys: 2min 14s, total: 27min 54s
Wall time: 3min 32s


GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster='gbtree',
                                    colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=0.5,
                                    gamma=None, gpu_id=None,
                                    importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=3, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=1000, n_job...
       0.1652, 0.1653, 0.1654, 0.1655, 0.1656, 0.1657, 0.1658, 0.1659,
       0.166 , 0.1661, 0.1662, 0.1663, 0.1664, 0.1665, 0.1666, 0.1667,
       0.1668, 0.1669, 0.167 , 0.1671, 0.1672, 0.1673, 0.1674, 0.1675,
       0.1676, 0.1677, 0.1678, 0.1679, 0.168 , 0.1681, 0.1682, 0.1683,
       0

In [20]:
gs_xgb.best_score_

0.9099893308585669

Reducing feature space

In [ ]:
importances = pd.Series(gs_xgb.best_estimator_.feature_importances_, index=train_features.columns).sort_values(ascending=False)
mask = (importances > 0.01)
selected_cols = importances[mask].index
selected_cols

In [ ]:
train_features_grid, train_features_validation, target_grid, target_validation = train_test_split(train_features, target, test_size = 0.1)

In [ ]:
np.random.seed(0)
fit_params={"early_stopping_rounds" : 10, 
            "eval_metric" : "mae", 
            "eval_set" : [[train_features_validation[selected_cols], target_validation]]}
%time gs_xgb.fit(train_features_grid[selected_cols], target_grid,verbose = 0, **fit_params)

In [ ]:
gs_xgb.best_score_

Fitting the residuals

In [ ]:
model_xgb = gs_xgb.best_estimator_
predvalues_train_xgb = pd.Series(model_xgb.predict(train_features))
prediction_train_xgb = pd.DataFrame({"Id" : train_data["Id"], "SalePrice" : predvalues_train_xgb})
prediction_train_xgb.set_index("Id", inplace = True)
residuals = target - prediction_train_xgb

In [ ]:
lm = Ridge()
lm_params = {"alpha" : np.linspace(0.001, 100, 100)}
gs_lm = GridSearchCV(lm, lm_params, cv = 10, return_train_score = True)

In [ ]:
def resid_correl(col):
    return pearsonr(residuals["SalePrice"], col)[0]

In [ ]:
correlations = [resid_correl(train_features[col]) for col in train_features.columns]
sorted_correlations = pd.Series(correlations, index=train_features.columns).sort_values(ascending=False)
mask = (sorted_correlations > 0.25) | (sorted_correlations < -0.25)
selected_cols = sorted_correlations[mask].index

In [ ]:
np.random.seed(0)
%time gs_lm.fit(train_features[selected_cols], residuals)

In [ ]:
gs_lm.best_score_

In [ ]:
gs_lm.best_estimator_.coef_

XGBoost outcomes

In [21]:
gs_xgb.best_params_

{'eta': 0.1522}

In [22]:
importances = pd.Series(gs_xgb.best_estimator_.feature_importances_, index=train_features.columns).sort_values(ascending=False)
importances[:20]

OverallQual      0.199939
KitchenQual      0.150754
GarageCars       0.107551
TotLivArea       0.100673
FullBath         0.066034
Neighborhood     0.051939
ExterQual        0.044741
BsmtExposure     0.037658
CentralAir       0.035097
HeatingQC        0.028861
Fireplaces       0.028569
TotRmsAbvGrd     0.016685
MSZoning         0.011873
ProxPos          0.011449
HalfBath         0.010504
AgeBuilt         0.009958
LotArea          0.009160
MSSubClass       0.008440
SaleCondition    0.007836
OverallCond      0.006977
dtype: float32

Running the model on the test dataset

In [24]:
model_xgb = gs_xgb.best_estimator_

In [25]:
pred_xgb = pd.Series(model_xgb.predict(test_features))
prediction_xgb = pd.DataFrame({"Id" : test_data["Id"], "SalePrice" : pred_xgb})
prediction_xgb.set_index("Id", inplace = True)

In [26]:
prediction_xgb.to_csv("./Data/prediction_xgb.csv")

Save encoder and model for later

In [ ]:
filename = 'fitted_model.sav'
joblib.dump(model_xgb, filename)
filename = 'fitted_encoder.sav'
joblib.dump(enc, filename)

In [ ]:
train_features.to_csv("./Data/encoded_train_data.csv")

In [ ]:
shap_values = shap.TreeExplainer(model_xgb).shap_values(train_features)

In [ ]:
shap.summary_plot(shap_values, train_features)

In [ ]:
shap.dependence_plot("TotLivArea", shap_values, train_features)

In [ ]:
shap.dependence_plot("OverallQual", shap_values, train_features)

In [ ]:
shap.dependence_plot("LotArea", shap_values, train_features)

In [ ]:
shap.dependence_plot("Neighborhood", shap_values, train_features)

In [ ]:
shap_interaction_values = shap.TreeExplainer(model_xgb).shap_interaction_values(train_features)

In [ ]:
shap.summary_plot(shap_interaction_values, train_features)

In [ ]:
shap.dependence_plot(
    ("TotLivArea", "TotLivArea"),
    shap_interaction_values, train_features,
    display_features = train_features
)

In [ ]:
shap.dependence_plot(
    ("TotLivArea", "OverallQual"),
    shap_interaction_values, train_features,
    display_features = train_features
)

In [ ]:
shap.dependence_plot(
    ("Neighborhood", "OverallQual"),
    shap_interaction_values, train_features,
    display_features = train_features
)

In [ ]:
tmp = np.abs(shap_interaction_values).sum(0)
for i in range(tmp.shape[0]):
    tmp[i,i] = 0
inds = np.argsort(-tmp.sum(0))[:50]
tmp2 = tmp[inds,:][:,inds]
pl.figure(figsize=(12,12))
pl.imshow(tmp2)
pl.yticks(range(tmp2.shape[0]), train_features.columns[inds], rotation=50.4, horizontalalignment="right")
pl.xticks(range(tmp2.shape[0]), train_features.columns[inds], rotation=50.4, horizontalalignment="left")
pl.gca().xaxis.tick_top()
pl.show()

In [ ]:
def heatmap(valuesDF, threshold):
    # Make a mask to only show the lower left part of the table
    mask = np.triu(np.ones_like(valuesDF, dtype = bool))

    cmap = sns.diverging_palette(220, 20, as_cmap = True)

    # Draw the heatmap with the mask and correct aspect ratio
    visual = sns.heatmap(valuesDF, mask = mask, cmap = cmap, center = threshold,
                square = True, linewidths = .5, annot = True);

    return visual